# Multi-Agents Supervisor Architecture – README 😊

## Обзор 🚀

Данный проект демонстрирует архитектуру многоагентной системы, где специальный супервайзер управляет работой различных агентов, отвечающих за генерацию, проверку тестовых сценариев и тестового кода. Система использует языковую модель AzureChatOpenAI и библиотеки для построения чат-агентов и графов состояний, обеспечивая автоматизацию сложного рабочего процесса. 👍

## Цель проекта 🎯

Основная цель проекта – организовать координацию работы агентов для создания и проверки тестовых сценариев и соответствующего им тестового кода. Супервайзер направляет поток задач между следующими ролями:
- **TestWriter.** Генерация тестовых сценариев.
- **TestReviewer.** Проверка и улучшение сценариев.
- **CodeWriter.** Написание тестового кода (на Python).
- **CodeReviewer.** Ревизия и улучшение кода.
- **Publisher.** Форматирование финального ответа для пользователя.

Супервайзер определяет, кто из агентов должен действовать следующим, и завершает работу, когда процесс выполнен. 😊

## Основные компоненты 🛠

### 1. Импорты и настройка окружения 🌐
- **Загрузка переменных окружения.** Использование `dotenv` и `os` для конфигурации проекта.
- **Библиотеки.** Импорт необходимых модулей для работы с типами данных, функциями, языковой моделью и построением графа состояний (StateGraph).

### 2. Функции для создания и обработки агентов 🤖
- **`create_agent`.** Функция создаёт агента с заданным системным промптом, объединяя шаблон чат-промпта с языковой моделью.
- **`agent_node`.** Функция обрабатывает узел агента, отправляя текущее состояние и получая результат. Для агента с именем "Publisher" дополнительно возвращается финальный результат.

### 3. Определение агентов и супервайзера 🔄
- **Агенты для тестовых сценариев и кода.** Создаются агенты для генерации и ревизии тестовых сценариев и кода: TestWriter, TestReviewer, CodeWriter, CodeReviewer.
- **Агент Publisher.** Отвечает за форматирование итогового ответа в формате Markdown.
- **Супервайзер.** Управляет диалогом между агентами. Супервайзер выбирает, кто должен действовать следующим, используя специальное определение функции `route`, связанное с JSON-выводом, и предоставляет выбор из списка ролей или завершение процесса (FINISH).

### 4. Построение графа состояний (StateGraph) 📊
- **Узлы графа.** Каждый агент (TestWriter, TestReviewer, CodeWriter, CodeReviewer, Publisher) и супервайзер являются узлами графа.
- **Маршрутизация.** Супервайзер направляет поток работы с помощью условных переходов: после каждого действия граф возвращается к супервайзеру, где на основе ответа выбирается следующий участник или завершение процесса.
- **Запуск графа.** Процесс начинается с супервайзера, который инициирует работу агентов, а затем происходит итеративное выполнение задач до получения финального результата.

### 5. Пример использования 💡
- **Цель.** Пример тестового кейса описывает задачу безопасного входа в приложение с использованием электронной почты и пароля, а также контроля доступа на основе ролей (администратор, пользователь, гость).
- **Обработка.** Пользовательский запрос передаётся в граф, где агенты последовательно генерируют тестовые сценарии и код. После этого финальный результат форматируется и выводится в Markdown.

## Как запустить проект 🚀

1. **Настройка окружения.** Создайте файл `.env` с необходимыми переменными, такими как `AZURE_OPENAI_ENDPOINT` и `AZURE_OPENAI_API_VERSION`.
2. **Установка зависимостей.** Убедитесь, что установлены все требуемые библиотеки (например, `dotenv`, `langchain`, `langgraph` и т.д.).
3. **Запуск в Jupyter Lab.** Откройте тетрадку в Jupyter Lab, выполните все ячейки последовательно. Граф, построенный системой, отобразится с использованием mermaid, а финальный результат будет выведен в консоли.

## Заключение ✨

Данная архитектура многоагентной системы демонстрирует, как с помощью супервайзера можно эффективно координировать работу различных агентов для создания и проверки тестовых сценариев и кода. Такой подход позволяет автоматизировать сложные рабочие процессы и повышать качество конечного продукта. Надеемся, что этот пример окажется полезным и вдохновит на дальнейшие эксперименты! 😊

## Load Env Vars

In [ ]:
# Импорт библиотек
from dotenv import load_dotenv
import os
from typing import Annotated, Sequence, TypedDict
import functools
import operator
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langgraph.graph import END, StateGraph, START

# Загрузка переменных окружения из файла .env
load_dotenv()

# Установка переменной окружения
os.environ["LANGCHAIN_PROJECT"] = "Multi-Agents-Supervisor-Architecture"

## Helper Utilities

In [ ]:
# Функция для создания агента
def create_agent(llm: AzureChatOpenAI, system_prompt: str) -> ChatPromptTemplate:
    """
    Description:
        Создает агента с заданным системным промптом.

    Args:
        llm: Модель языка AzureChatOpenAI.
        system_prompt: Системный промпт для агента.

    Returns:
        Шаблон чат-промпта для агента.

    Examples:
        >>> llm = AzureChatOpenAI(...)
        >>> system_prompt = "You are an assistant."
        >>> create_agent(llm, system_prompt)
    """
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    return prompt | llm

In [ ]:
# Функция для обработки узла агента
def agent_node(state: dict, agent: ChatPromptTemplate, name: str) -> dict:
    """
    Description:
        Обрабатывает узел агента и возвращает результат.

    Args:
        state: Состояние для передачи агенту.
        agent: Агент для обработки состояния.
        name: Имя агента.

    Returns:
        Словарь с сообщениями и финальным результатом (если есть).

    Examples:
        >>> state = {"messages": [...] }
        >>> agent = create_agent(...)
        >>> agent_node(state, agent, "Publisher")
    """
    result = agent.invoke(state)

    if name == "Publisher":
        return {
            "messages": [HumanMessage(content=result.content, name=name)],
            "final_result": result.content
        }
    return {
        "messages": [HumanMessage(content=result.content, name=name)]
    }

### Create Agent Supervisor


In [ ]:
# Инициализация переменных
deployment_name = 'gpt-4o-mini'

members = ["TestWriter", "TestReviewer", "CodeWriter", "CodeReviewer", "Publisher"]

system_prompt = (
    "Вы являетесь супервайзером, вам поручено управлять разговором между "
    "следующими работниками: {members}. Учитывая запрос пользователя, "
    "ответьте, кто из работников должен действовать следующим. "
    "Каждая выполненная работа должна быть проверена соответствующим "
    "ревьюером, если таковой имеется. Каждый работник выполнит задачу и "
    "ответит своими результатами и статусом. Перед завершением опубликуйте "
    "ответ и после этого ответьте 'FINISH'."
)
options = ["FINISH"] + members

# Определение функции
function_def = {
    "name": "route",
    "description": "Выберите следующую роль.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

# Создание шаблона чат-промпта
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Учитывая вышеуказанный разговор, кто должен действовать следующим? "
            "Или мы должны завершить? Выберите один из вариантов: {options}"
        ),
    ]
).partial(options=str(options), members=", ".join(members))

# Инициализация модели языка
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=deployment_name,
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

# Создание цепочки супервайзера
supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

### Construct Graph


In [ ]:
# Определение языка кода
code_language = "Python"

# Определение типа данных для состояния агента
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str
    final_result: str

# Создание агентов
test_scenario_writer_agent = create_agent(llm,
                                          """
                                          Вы должны предоставить набор связных и
                                          хорошо определенных тестовых сценариев
                                          для другого агента для написания кодов,
                                          или, если это необходимо, улучшить
                                          существующие тестовые сценарии в
                                          соответствии с данными инструкциями.
                                          Вы не должны предоставлять коды или
                                          другие функции, которые не являются
                                          вашей основной целью.
                                          """
                                          )

test_scenario_reviewer_agent = create_agent(llm,
                                            """
                                            Вы должны проверить набор тестовых
                                            сценариев, и если тестовые сценарии
                                            нуждаются в улучшении, вы должны
                                            предоставить инструкции о том, как
                                            их улучшить. Вы не должны
                                            предоставлять коды или другие
                                            функции, которые не являются вашей
                                            основной целью.
                                            """
                                            )

test_code_writer_agent = create_agent(llm, f"""Вы должны писать {code_language} код для тестовых сценариев""")

test_code_reviewer_agent = create_agent(llm,
                                        """
                                        Вы должны проверить {code_language} код,
                                        и если код нуждается в улучшении, вы
                                        должны предоставить инструкции о том,
                                        как его улучшить. Вы не должны
                                        предоставлять коды или другие функции,
                                        которые не являются вашей основной
                                        целью.
                                        """
                                        )

publisher_agent = create_agent(llm,
                               """
                               Вы должны отформатировать финальный ответ в
                               формате markdown для пользователя. Ответ должен
                               содержать финальную версию тестовых сценариев и
                               финальную версию кода.
                               """
                               )

# Создание узлов агентов
test_scenario_writer_node = functools.partial(agent_node, agent=test_scenario_writer_agent, name="TestWriter")
test_scenario_reviewer_node = functools.partial(agent_node, agent=test_scenario_reviewer_agent, name="TestReviewer")
test_code_writer_node = functools.partial(agent_node, agent=test_code_writer_agent, name="CodeWriter")
test_code_reviewer_node = functools.partial(agent_node, agent=test_code_reviewer_agent, name="CodeReviewer")
publisher_node = functools.partial(agent_node, agent=publisher_agent, name="Publisher")

# Создание рабочего процесса
workflow = StateGraph(AgentState)
workflow.add_node("TestWriter", test_scenario_writer_node)
workflow.add_node("TestReviewer", test_scenario_reviewer_node)
workflow.add_node("CodeWriter", test_code_writer_node)
workflow.add_node("CodeReviewer", test_code_reviewer_node)
workflow.add_node("Publisher", publisher_node)
workflow.add_node("supervisor", supervisor_chain)

In [ ]:
# Добавление ребер в граф
for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)

workflow.add_edge(START, "supervisor")

In [ ]:
# Компиляция графа
graph = workflow.compile()
for k, v in conditional_map.items():
    print(f"{k} -> {v}")

In [ ]:
# Отображение графа
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    pass

In [ ]:
# Определение цели
objective = """
    Как пользователь, я хочу безопасно войти в приложение, используя свой
    адрес электронной почты и пароль, чтобы получить доступ к своей
    персонализированной панели управления и функциям. Система также должна
    поддерживать контроль доступа на основе ролей, чтобы ограничить доступ к
    определенным областям приложения в зависимости от ролей пользователей
    (например, администратор, пользователь, гость).
"""

# Инициализация финального результата
final_result = ""

# Обработка потока графа
for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content=f"Я хочу, чтобы вы написали 3 тестовых сценария для "
                        f"следующей цели: {objective} для каждого тестового "
                        f"сценария я хочу {code_language} скрипт кода. "
                        "Как только вы напишете код, завершите."
            )
        ]
    }
):
    if "__end__" not in s:
        print("#" * 50)
        for k, v in s.items():
            print(f"{k} : \n")
            if "messages" in v:
                print(" - messages :", v.get("messages")[-1].content)
            if "next" in v:
                print(" - next : ", v.get("next"))
            if "final_result" in v:
                final_result = v.get("final_result")
        print()

## Display Final Results of the Publisher

In [ ]:
print(final_result)